[View in Colaboratory](https://colab.research.google.com/github/sungreong/Pymc3_bayseian/blob/master/Gaussian_Processes.ipynb)

In [0]:
!pip install git+https://github.com/pymc-devs/pymc3

## Reference https://docs.pymc.io/gp.html

## GP Basics 

파라미터와 변수를 모델에서 알지 못할 때  그것들은 상수나 고정된 벅테가 아니라 **함수**가 된다.  
**Gaussian process(GP)** 는 연속전  함수들의 공간에 대한 정의역으로 가지는 사전 확률 분포로 사용 되어진다.  
**f(x) 에 대한 GP prior**는 보통 $ f(x) \sim gp(m(x) , k(x, x^{,} ) $ 

함수 값은 다둥 노말 분포로부터 뽑은 값이다. 평균 함수를 피라미라터 가진다. $m(x) and covariance function k(x, x^{''}) $  
가우시안 프로세스들은 marginalization 때문에 함수에 대한 사전분포 선택이 편리하다.  
보통 f(x) 에 대한 marginalization 분포는 **inference step** 동안 평가 되어진다.  
조건부분포는 새로운 데이터 $x_* 에 대한 새로운 함수  f(x_*)$를 예측하는데 사용된다.

![가우시안 분포](https://preview.ibb.co/dGPvyy/image.png)


pymc3 는 완전한 베이지안 가우시안 프로세스 모델을 작업하기에 좋은 환경이다

명백한 문법과 높은 구성도와 미리 정의된 covariance functions , mean functions and 몇몇 GP 시행하는 것들이 포함되어 있다.   
GP는 크거나 위계가 있는 모델 안에서 다루어지고 홀로 이루어진 모델로써 되지 않는다.

## Mean functions and Covariance functions  
GPy , GPflow 패키지지가 mean function 과 covariance function 구조를 가진 문법 패키지 들이 있다.  
처음 객체화 할때 mean 과 covariance 함수들은 파라미터화 한다.  그러나 인풋이 아직 주어진 것이 아니다.  
covariance function은 반드시 추가적으로 인풋 메트릭스 안에서 큰 차원이 추가 되어야하고,   
이러한 디자인의 이유는 covariance 함수가 다른 covariance 함수들과 결합이 된 구조를 이뤄야 하기 때문이다.


예를 들어, 3 개의 예측 변수를 나타내는 3 열 행렬의 두 번째 및 세 번째 열에서 작동하는 지수화 된 2 차 공분산 함수를 생성하려면 다음을 수행합니다


여기에서 ls 또는 lengthscale 매개 변수는 2 차원이므로 두 번째 및 세 번째 차원의 길이가 서로 다릅니다.  

The reason we have to specify **input_dim**, the total number of columns of X,   
and **active_dims**, which of those columns or dimensions the covariance function will act on, is because **cov_func hasn’t actually seen the input data yet**  
**active_dims** argument is optional, and defaults to all columns of the matrix of inputs.


## sum, product covariance functions
```
cov_func = pm.gp.cov.ExpQuad(...) + pm.gp.cov.ExpQuad(...)
cov_func = pm.gp.cov.ExpQuad(...) * pm.gp.cov.Periodic(...)
cov_func = eta**2 * pm.gp.cov.Matern32(...)
```

after the covariance function is defined ,  conv_func(), (mean_func()) 을 호출해서 계산한다.




In [0]:
import pymc3 as pm 
ls = [2, 5] # the lengthscales
cov_func = pm.gp.cov.ExpQuad(input_dim=3, ls=ls, active_dims=[1, 2])

## GP Implementations

PyMC3 includes several GP implementations, including marginal and latent variable models and also some fast approximations

1. First, a GP is instantiated with a mean function and a covariance function.
2. Then, GP objects can be added together, allowing for function characteristics to be carefully modeled and separated. 
3. Finally, **one of prior, marginal_likelihood or conditional methods** is called on the GP object to actually construct the PyMC3 random variable that represents the function prior.



```
gp = pm.gp.Latent(mean_func, cov_func)
```

## Note
The gp.Marginal class and similar don’t have a $ prior $ method.   
Instead they have a $ marginal \ likelihood $ method that is used similarly,   
but has additional required arguments, such as the observed data, noise, or other, depending on the implementation.   
See the notebooks for examples.$  The \ conditional \ method $ works similarly.


```
gp = pm.gp.Latent(mean_func, cov_func)
Calling the prior method will create a PyMC3 random variable that represents the latent function f(x)=f:
f = gp.prior("f", X)
```

###  f is a random variable that can be used within a PyMC3 model like any other type of random variable.
1. The first argument is the name of the random variable representing the function we are placing the prior over
2.  The second argument is the inputs to the function that the prior is over, X. The inputs are usually known and present in the data, but they can also be PyMC3 random variables. 
3. If the inputs are a Theano tensor or a PyMC3 random variable, the shape needs to be given.


 inference is performed on the model.   
 The  $ conditional \ method $ creates the conditional, or predictive, distribution over the latent function at arbitrary x∗ input points, $ f(x_∗). $To construct the conditional distribution we write:

```
f_star = gp.conditional("f_star", X_star)
```

## Additive GPs

```
gp1 = pm.gp.Marginal(mean_func1, cov_func1)
gp2 = pm.gp.Marginal(mean_func2, cov_func2)
gp3 = gp1 + gp2
```
![대체 텍스트](https://preview.ibb.co/jgj1oy/gpgp.png/)



```
with pm.Model() as model:
    gp1 = pm.gp.Marginal(mean_func1, cov_func1)
    gp2 = pm.gp.Marginal(mean_func2, cov_func2)

    # gp represents f1 + f2.
    gp = gp1 + gp2

    f = gp.marginal_likelihood("f", X, y, noise)

    trace = pm.sample(1000)
 ```
 To construct the conditional distribution of gp1 or gp2, we also need to include the additional arguments, X, y, and noise:
 ```
  
 with model:
    # conditional distributions of f1 and f2
    f1_star = gp1.conditional("f1_star", X_star,
                              given={"X": X, "y": y, "noise": noise, "gp": gp})
    f2_star = gp2.conditional("f2_star", X_star,
                              given={"X": X, "y": y, "noise": noise, "gp": gp})

    # conditional of f1 + f2, `given` not required
    f_star = gp.conditional("f_star", X_star)
```



This second block produces the conditional distributions.   
Notice that extra arguments are required for conditionals of f1 and f2, but not f.  
This is because those arguments are cached when .**marginal_likelihood** is called on **gp.**


## Note

When constructing conditionals, the additional arguments X, y, noise and gp must be provided as a dict called given!


Since the marginal likelihoood method of **gp1 or gp2 weren’t called**, **their conditionals** need to be provided with **the required inputs.**   
In the same fashion as the prior, f_star, f1_star and f2_star are random variables that can now be used like any other random variable in PyMC3.  
Check the notebooks for detailed demonstrations of the usage of GP functionality in PyMC3.


NameError: ignored